In [1]:
import torch
import tensorflow as tf
import pprint
from datasets import load_dataset
import openai
from IPython.display import display, HTML
import time
import re
import numpy as np
import pandas as pd


2025-05-09 22:01:46.579861: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-09 22:01:46.589187: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-09 22:01:46.598552: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-09 22:01:46.612733: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-09 22:01:46.616992: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attemptin

In [2]:
print(torch.cuda.is_available())  # Should return True if CUDA is available

True


In [3]:
current_device = torch.cuda.current_device()
print(f"Current device ID: {current_device}")
print(f"Current device name: {torch.cuda.get_device_name(current_device)}")


Current device ID: 0
Current device name: NVIDIA A40-24Q


In [4]:
import os
print("Current directory:", os.getcwd())  # Currently: /scratch/h/Hassan.Mo/LLM/Examples/Videos

# Go up to the directory containing your file
os.chdir("/scratch/h/Hassan.Mo/LLM/")  # Absolute path (recommended)
# OR: os.chdir("../../")  # Relative path (go up two levels)

print("New directory:", os.getcwd())
print("Files here:", os.listdir())  # Check if open_ai_API.txt is listed

Current directory: /scratch/h/Hassan.Mo/LLM
New directory: /scratch/h/Hassan.Mo/LLM
Files here: ['hybrid_loop.ipynb', 'media', 'prompt_exp.ipynb', '.ipynb_checkpoints', 'Examples', 'final_set.csv', 'animation_check.ipynb', 'open_ai_API.txt', '=3.2.0', 'overlap_investigation.ipynb', 'sample_prompt_gen.ipynb', 'hybrid.ipynb', 'looping.ipynb', 'requirments.txt', 'anomaly_solver.ipynb', 'utils.py', '__pycache__', 'Steffan_results', 'animation_checker.py', 'motion_set.csv', 'log_text_condensed.py', '.vscode']


In [5]:
# Reading API Key from file
API_KEY = open("open_ai_API.txt", "r").read().strip()
openai.api_key = API_KEY

In [6]:
from utils import llm_tools, tools_local
lt = llm_tools()
tl = tools_local()

In [8]:
def prompt_gen(formatted_code_string, model="gpt-4o-mini", max_tokens=400, temperature=0.01):
        """
        Prepares messages for the Chat API and sends a request.

        Args:
            formatted_code_string (str): The code snippet to be analyzed.
            model (str): The model to use for the API call. Defaults to "gpt-4o-mini".
            max_tokens (int): The maximum number of tokens in the response. Defaults to 150.
            temperature (float): Sampling temperature. Defaults to 0.01.

        Returns:
            dict: The response from the OpenAI API, or None if an error occurs.
        """
        # Prepare the messages for the Chat API
        messages = [
            {"role": "user", "content": f"You are a helpful assistant which would suggest a prompt from the user which would generate such a code: {formatted_code_string}, Make sure the language of the prompt is very natural"}]

        # Make the API call
        try:
            response = openai.chat.completions.create(
                model=model,
                messages=messages,
                max_tokens=max_tokens,
                temperature=temperature
            )
            return response
        except Exception as e:
            print(f"Error occurred while making the API call: {e}")
            return None

In [9]:


def read_python_files(directory):
    """
    Reads the content of all Python (.py) files in a given directory.

    Args:
        directory (str): The path to the directory containing the Python files.

    Returns:
        list: A list where each element is the content of a Python file found in the directory.
              Returns an empty list if no Python files are found or if an error occurs.
    """
    python_files_content = []

    try:
        # Get all .py files in the directory
        py_files = [f for f in os.listdir(directory) if f.endswith('.py')]

        if not py_files:
            print("No Python files found in the directory.")
        else:
            print(f"Found {len(py_files)} Python files:")

            for filename in py_files:
                file_path = os.path.join(directory, filename)

                print(f"\nReading: {filename}")
                with open(file_path, 'r') as f:
                    content = f.read()
                    python_files_content.append(content)  # Store content in array
                    print(f"Stored content (length: {len(content)} chars)")

            print(f"\nTotal files stored: {len(python_files_content)}")

    except FileNotFoundError:
        print(f"Directory not found: {directory}")
    except PermissionError:
        print(f"Permission denied for: {directory}")
    except Exception as e:
        print(f"An error occurred: {str(e)}")

    return python_files_content



In [13]:

def generate_and_extract(file_contents):
    """
    Generates responses for code content and then extracts quoted text from those responses.

    Args:
        file_contents (list): A list of strings, where each string is the content of a code file.

    Returns:
        list: A list of strings, where each string contains all the extracted quoted text
              joined by spaces from the generated response for the corresponding code content.
    """
    all_extracted_texts = []
    for i, content in enumerate(file_contents):
        print(f"\nProcessing code {i}:")
        response = prompt_gen(content)
        response_text = response.choices[0].message.content
        # tl.printer(response_text) # Uncomment to print each response
        print(f"Generated response (length: {len(response_text)} chars)")

        # Extract quoted text from the generated response
        matches = re.findall(r'"(.*?)"|\'(.*?)\'', response_text)
        flattened_matches = [match[0] or match[1] for match in matches if any(match)]
        extracted_text = ' '.join(flattened_matches)
        all_extracted_texts.append(extracted_text)
        print(f"Extracted text (length: {len(extracted_text)} chars): {extracted_text[:100]}...") # Print a snippet

    print(f"\nExtracted quoted text from {len(all_extracted_texts)} responses.")
    return all_extracted_texts


In [14]:
# Example usage:
directory_videos = "/scratch/h/Hassan.Mo/LLM/Examples/Videos/"   # Replace with correct address for both
directory_static = "/scratch/h/Hassan.Mo/LLM/Examples/Static/"

file_contents = read_python_files(directory_static)              # Change input variable accordingly
extracted_quoted_texts = generate_and_extract(file_contents)

# file_contents = read_python_files(directory_static)

# Access stored contents later
# if file_contents:
    # print("\nSample of first file content:")                                                    # Uncomment to print results
    # print(file_contents[0][:200] + "...")  # Show first 200 chars of first file


# Assuming 'file_contents' is already populated from the read_python_files function


# Now 'extracted_quoted_texts' contains the text within quotes from each generated response
# if extracted_quoted_texts:
#     print("\nExtracted quoted texts from all responses:")
#     for i, text in enumerate(extracted_quoted_texts):
#         print(f"Response {i+1}: {text}")                                                      # Uncomment to print results

Found 10 Python files:

Reading: example_22.py
Stored content (length: 638 chars)

Reading: example_21.py
Stored content (length: 357 chars)

Reading: example_18.py
Stored content (length: 673 chars)

Reading: example_25.py
Stored content (length: 884 chars)

Reading: example_2.py
Stored content (length: 465 chars)

Reading: example_16.py
Stored content (length: 960 chars)

Reading: example_3.py
Stored content (length: 370 chars)

Reading: example_1.py
Stored content (length: 688 chars)

Reading: example_14.py
Stored content (length: 1224 chars)

Reading: example_4.py
Stored content (length: 390 chars)

Total files stored: 10

Processing code 0:
Generated response (length: 400 chars)
Extracted text (length: 309 chars): Can you help me create a 3D scene using Manim that includes a set of axes and a sphere? I want the s...

Processing code 1:
Generated response (length: 415 chars)
Extracted text (length: 336 chars): Can you help me create a simple 3D scene using Manim? I want to include 

In [15]:
final_set = np.column_stack((extracted_quoted_texts, file_contents))
# Convert to DataFrame
df = pd.DataFrame(final_set)

# Export to CSV
df.to_csv('static.csv', index=False)  # `index=False` avoids extra row numbers

In [23]:
from IPython.display import display, HTML
display(HTML(df.to_html()))



,0,1
0,"Can you help me create a 3D scene using Manim that includes a set of axes and a sphere? I want the sphere to have a checkerboard texture in shades of red, and I’d like to position the light source to come from behind the scene. Also, please set the camera orientation to give a nice perspective of the sphere.","from manim import *\n\nclass ThreeDLightSourcePosition(ThreeDScene):\n def construct(self):\n axes = ThreeDAxes()\n sphere = Surface(\n lambda u, v: np.array([\n 1.5 * np.cos(u) * np.cos(v),\n 1.5 * np.cos(u) * np.sin(v),\n 1.5 * np.sin(u)\n ]), v_range=[0, TAU], u_range=[-PI / 2, PI / 2],\n checkerboard_colors=[RED_D, RED_E], resolution=(15, 32)\n )\n self.renderer.camera.light_source.move_to(3*IN) # changes the source of the light\n self.set_camera_orientation(phi=75 * DEGREES, theta=30 * DEGREES)\n self.add(axes, sphere)\n"
1,"Can you help me create a simple 3D scene using Manim? I want to include a 3D text that says 'This is a 3D text' and have it fixed in the frame while also displaying some 3D axes. I’d like the camera to be oriented at a specific angle, with the text positioned in the upper left corner of the screen. Could you provide the code for that?","from manim import *\n\nclass FixedInFrameMObjectTest(ThreeDScene):\n def construct(self):\n axes = ThreeDAxes()\n self.set_camera_orientation(phi=75 * DEGREES, theta=-45 * DEGREES)\n text3d = Text(""This is a 3D text"")\n self.add_fixed_in_frame_mobjects(text3d)\n text3d.to_corner(UL)\n self.add(axes)\n self.wait()\n"
2,"Can you help me create a simple heat diagram plot using Manim? I want to include axes labeled with ΔQ and temperature in degrees Celsius. The x-axis should range from 0 to 40, and the y-axis should range from -8 to 32. I also need to plot a line graph with specific points: (0, 20), (8, 0), (38, 0), and (39, -5). Please make sure the graph looks clear and professional!","from manim import *\n\nclass HeatDiagramPlot(Scene):\n def construct(self):\n ax = Axes(\n x_range=[0, 40, 5],\n y_range=[-8, 32, 5],\n x_length=9,\n y_length=6,\n x_axis_config={""numbers_to_include"": np.arange(0, 40, 5)},\n y_axis_config={""numbers_to_include"": np.arange(-5, 34, 5)},\n tips=False,\n )\n labels = ax.get_axis_labels(\n x_label=Tex(r""$\Delta Q$""), y_label=Tex(r""T[$^\circ C$]"")\n )\n\n x_vals = [0, 8, 38, 39]\n y_vals = [20, 0, 0, -5]\n graph = ax.plot_line_graph(x_values=x_vals, y_values=y_vals)\n\n self.add(ax, labels, graph)\n"
3,"Can you help me create a 3D surface plot using Manim? I want to visualize a Gaussian function over a range of values. The surface should be colorful, with a checkerboard pattern, and I’d like to set the camera orientation to get a good view of the plot. Also, please include 3D axes for reference.","from manim import *\n\nclass ThreeDSurfacePlot(ThreeDScene):\n def construct(self):\n resolution_fa = 24\n self.set_camera_orientation(phi=75 * DEGREES, theta=-30 * DEGREES)\n\n def param_gauss(u, v):\n x = u\n y = v\n sigma, mu = 0.4, [0.0, 0.0]\n d = np.linalg.norm(np.array([x - mu[0], y - mu[1]]))\n z = np.exp(-(d ** 2 / (2.0 * sigma ** 2)))\n return np.array([x, y, z])\n\n gauss_plane = Surface(\n param_gauss,\n resolution=(resolution_fa, resolution_fa),\n v_range=[-2, +2],\n u_range=[-2, +2]\n )\n\n gauss_plane.scale(2, about_point=ORIGIN)\n gauss_plane.set_style(fill_opacity=1,stroke_color=GREEN)\n gauss_plane.set_fill_by_checkerboard(ORANGE, BLUE, opacity=0.5)\n axes = ThreeDAxes()\n self.add(axes,gauss_plane)\n"
4,"Can you help me create a Manim scene that illustrates the concept of horizontal distance between two points? I want to include two dots at specific coordinates, connect them with a line, and add braces with annotations. The first brace should label the line as 'Horizontal distance', and the second brace should represent the difference in the x-coordinates with the label 'x - x₁'. Please make sure the line is colored orange and the braces are positioned correctly.","from manim import *\n\nclass BraceAnnotation(